# Smoothing
-----------

This notebook shows how to use `dysh` to smooth and decimate a spectrum.   We will use data from the Position-Switch example.

In [ ]:
import os
import wget
import astropy.units as u
from dysh.fits.gbtfitsload import GBTFITSLoad

In [ ]:
# show the mean and std for data and for a 1-rolled data
def rolled_stats(data):
    delta = data[1:] - data[:-1]
    print('stats:',data.mean(), data.std(), delta.mean(), delta.std())

# to test, check that rolled_stats has a sqrt(2) higher RMS
rolled_stats(np.random.normal(0,1,10000))

## We use the FITS file already used in the Position-Switch example

this should be as simple as:

       filename = get_dysh_data(test='getps')


In [ ]:
filename = "TGBT21A_501_11.raw.vegas.fits"
if not os.path.isfile(filename):
    url = f"http://www.gb.nrao.edu/dysh/example_data/onoff-L/data/{filename}"
    print(f"Downloading {filename}")
    wget.download(url,out=filename)
    print(f"\nRetrieved {filename}")
else:
    print(f"{filename} already downloaded")

In [ ]:
sdfits = GBTFITSLoad(filename)

## Get a time-averaged spectrum at the highest resolution


In [ ]:
psscan = sdfits.getps(scan=152, ifnum=0, plnum=0)
ta = psscan.timeaverage(weights='tsys')
ta.meta['CDELT1'] = 1000.0
ta.plot(xaxis_unit="chan",yaxis_unit="mK",ymin=-100,ymax=500, grid=True)
rolled_stats(ta.flux[21000:28000])

## Smooth in a few ways

In [ ]:
ts1 = ta.smooth('gaussian',16)
ts1.plot(xaxis_unit="chan",yaxis_unit="mK",ymin=-100,ymax=500,grid=True)
rolled_stats(ts1.flux[21000//16:28000//16])

In [ ]:
ts2 = ta.smooth('box', 320)
ts2.plot(xaxis_unit="chan",yaxis_unit="mK",ymin=-100,ymax=500, grid=True)

In [ ]:
ta.plot(xaxis_unit="km/s",yaxis_unit="mK",ymin=-100,ymax=500)

In [ ]:
ts2.plot(xaxis_unit="km/s",yaxis_unit="mK",ymin=-100,ymax=500)
print(ts2._meta['CDELT1'])

## Successive smoothing check

In [ ]:
ts5a = ta.smooth('gaussian',3,-1).smooth('gaussian',5,-1)
ts5b = ta.smooth('gaussian',5,-1)
(ts5a-ts5b).plot()
rolled_stats((ts5a-ts5b).flux[21000:28000])

In [ ]:
ta2 = 0.5 * (ta + ta)

In [ ]:
ts5b = ta.smooth('gausn',5,-1)